In [1]:
import pandas as pd
import numpy as np
import koreanize_matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import timedelta
import ast

# 하트 수, 친구 수 이상치 기준 확인해보기

In [2]:
accounts_user = pd.read_csv('./dumps/processed_csv/processed_accounts_user.csv')
accounts_paymenthistory = pd.read_csv('./dumps/processed_csv/processed_accounts_paymenthistory.csv')
accounts_failpaymenthistory = pd.read_csv('./dumps/processed_csv/processed_accounts_failpaymenthistory.csv')

hackle_merge = pd.read_csv('./dumps/processed_csv/processed_hackle_merge.csv')

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_8372\3035445703.py:5: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  hackle_merge = pd.read_csv('./dumps/processed_csv/processed_hackle_merge.csv')


## user_id 처리 (int -> str)

In [3]:
hackle_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20203847 entries, 0 to 20203846
Data columns (total 16 columns):
 #   Column          Dtype  
---  ------          -----  
 0   session_id      object 
 1   user_id         float64
 2   language        object 
 3   osname          object 
 4   osversion       object 
 5   versionname     object 
 6   device_id       object 
 7   event_id        object 
 8   event_datetime  object 
 9   event_key       object 
 10  item_name       object 
 11  page_name       object 
 12  friend_count    float64
 13  votes_count     float64
 14  heart_balance   float64
 15  question_id     float64
dtypes: float64(5), object(11)
memory usage: 2.4+ GB


In [ ]:
accounts_user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 677080 entries, 0 to 677079
Data columns (total 15 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   user_id             677080 non-null  int64  
 1   gender              677080 non-null  object 
 2   point               677080 non-null  int64  
 3   friend_id_list      674210 non-null  object 
 4   is_push_on          677080 non-null  int64  
 5   created_at          677080 non-null  object 
 6   block_user_id_list  677080 non-null  object 
 7   hide_user_id_list   677080 non-null  object 
 8   ban_status          677080 non-null  object 
 9   report_count        677080 non-null  int64  
 10  alarm_count         677080 non-null  int64  
 11  pending_chat        677080 non-null  int64  
 12  pending_votes       677080 non-null  int64  
 13  group_id            677080 non-null  float64
 14  friend_count        677080 non-null  int64  
dtypes: float64(1), int64(8), object(6)

In [8]:
accounts_paymenthistory.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95140 entries, 0 to 95139
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     95140 non-null  int64 
 1   productId   95140 non-null  object
 2   phone_type  95140 non-null  object
 3   created_at  95140 non-null  object
dtypes: int64(1), object(3)
memory usage: 2.9+ MB


In [9]:
accounts_failpaymenthistory.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160 entries, 0 to 159
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          160 non-null    int64 
 1   user_id     160 non-null    int64 
 2   productId   56 non-null     object
 3   phone_type  160 non-null    object
 4   created_at  160 non-null    object
dtypes: int64(2), object(3)
memory usage: 6.4+ KB


In [3]:
# user_id 개별 처리 (str로 바꿔줌)

# hackle_merge는 user_id에 null값이 포함되어 있어서 pandas 전용 int 형변환 Int64로 해줌
hackle_merge['user_id'] = hackle_merge['user_id'].astype('Int64').astype(str)  
accounts_user['user_id'] = accounts_user['user_id'].astype(str)
accounts_paymenthistory['user_id'] = accounts_paymenthistory['user_id'].astype(str)
accounts_failpaymenthistory['user_id'] = accounts_failpaymenthistory['user_id'].astype(str)

In [ ]:
# 잘 처리됐는지 확인
hackle_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20203847 entries, 0 to 20203846
Data columns (total 16 columns):
 #   Column          Dtype  
---  ------          -----  
 0   session_id      object 
 1   user_id         object 
 2   language        object 
 3   osname          object 
 4   osversion       object 
 5   versionname     object 
 6   device_id       object 
 7   event_id        object 
 8   event_datetime  object 
 9   event_key       object 
 10  item_name       object 
 11  page_name       object 
 12  friend_count    float64
 13  votes_count     float64
 14  heart_balance   float64
 15  question_id     float64
dtypes: float64(4), object(12)
memory usage: 2.4+ GB


## hackle 유저의 하트 수 이상치 기준 확인

In [11]:
hackle_merge.head(3)

,session_id,user_id,language,osname,osversion,versionname,device_id,event_id,event_datetime,event_key,item_name,page_name,friend_count,votes_count,heart_balance,question_id
0,8QXy31PQxbW9qLzq0Y1dhR8Ypm52,1046711,ko-KR,iOS,16.5.1,2.0.3,D5417226-F71B-4A9E-A180-CD072F2AB279,00000716-27e9-4e72-a602-d0ce61784b06,2023-07-18 21:07:24,click_question_open,NaN,NaN,64.0,436.0,4830.0,NaN
1,8QXy31PQxbW9qLzq0Y1dhR8Ypm52,1046711,ko-KR,iOS,16.5.1,2.0.3,D5417226-F71B-4A9E-A180-CD072F2AB279,0294d883-0e06-415c-b970-5e8ee2517b60,2023-07-18 21:01:38,$session_start,NaN,NaN,NaN,NaN,NaN,NaN
2,8QXy31PQxbW9qLzq0Y1dhR8Ypm52,1046711,ko-KR,iOS,16.5.1,2.0.3,D5417226-F71B-4A9E-A180-CD072F2AB279,0acb0eb0-3bc9-48b9-a166-080888572fc6,2023-07-18 21:17:44,view_lab_tap,NaN,NaN,64.0,436.0,4830.0,NaN


In [4]:
# hackle에 찍힌 유저들만 추출
hackle_users = hackle_merge['user_id'].unique()

# 날짜 범위 확인
hackle_merge['event_datetime'].min(), hackle_merge['event_datetime'].max()

('2023-07-18 00:00:00', '2023-08-10 23:59:59')

In [5]:
# hackle 유저로 필터링
user_hackle = accounts_user[accounts_user['user_id'].isin(hackle_users)]
payment_hackle = accounts_paymenthistory[accounts_paymenthistory['user_id'].isin(hackle_users)]
failpayment_hackle = accounts_failpaymenthistory[accounts_failpaymenthistory['user_id'].isin(hackle_users)]

# 실패 기록의 id 제거
failpayment_hackle.drop(columns='id', inplace=True)

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_8372\3590997011.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  failpayment_hackle.drop(columns='id', inplace=True)


In [6]:
# 근데 어떤 유저들은 해클 기간 훨씬 이전에 가입해서
# 해클 기간까지 구매를 해서 하트가 쌓였을 수도 있지 않나?

# 기간으로도 필터링하기에는 안 맞지 않나..?
# 그럼 날짜 max로만 자르자!
payment_date_cond = payment_hackle['created_at'] <= hackle_merge['event_datetime'].max()
failpayment_date_cond = failpayment_hackle['created_at'] <= hackle_merge['event_datetime'].max()

# 해클 max 기간 이전 구매 기록들
payment_hackle = payment_hackle[payment_date_cond]
failpayment_hackle = failpayment_hackle[failpayment_date_cond]

In [50]:
# 잘 잘렸나 확인
origin_payment_date = accounts_paymenthistory['created_at'].max()
edited_payment_date = payment_hackle['created_at'].max()

print(f'오리지널 paymenthistory: {origin_payment_date}\
      \n날짜 변경 paymenthistory: {edited_payment_date}')

오리지널 paymenthistory: 2024-05-08 23:12:45      
날짜 변경 paymenthistory: 2023-08-10 21:33:52


In [ ]:
# 상품명에 적힌 하트 수 컬럼명 생성
# 취소기록은 아직 합치지 않은 상태에서 총 구매 하트를 확인해보려고 함
buy_hearts_hackle = payment_hackle.copy()
buy_hearts_hackle['hearts'] = buy_hearts_hackle['productId'].apply(lambda x: int(x.split('.')[-1]))

# 구매 기록의 총 하트 수 (취소 기록 제외 X)
buy_hearts_hackle.groupby('user_id')['hearts'].sum().reset_index(name='total_hearts').sort_values(by='total_hearts', ascending=False)

,user_id,total_hearts
4758,1141603,75977
5790,1167602,56777
8149,1216899,49977
3654,1111139,44777
2579,1081092,38977
...,...,...
3873,1117928,200
3622,1110744,200
2989,1093082,200
2655,1083594,200


### productId null 분석

In [36]:
notnull_fail = len(failpayment_hackle[failpayment_hackle['productId'].notnull()])
null_fail = len(failpayment_hackle[failpayment_hackle['productId'].isnull()])

print(f'null이 아닌 실패기록: {notnull_fail}\
      \nnull 실패기록: {null_fail}')


null이 아닌 실패기록: 22      
null 실패기록: 63


In [10]:
payment = payment_hackle.copy()
failpayment = failpayment_hackle.copy()
failpayment

# 합쳤을 때 구분을 위해 표시해줌
payment['is_success'] = 's'
failpayment['is_success'] = 'f'

# 모든 기록을 합침
total_history = pd.concat([payment, failpayment])
total_history = total_history.sort_values(by=['user_id', 'created_at']).reset_index()
total_history

,index,user_id,productId,phone_type,created_at,is_success
0,37513,1000061,heart.777,I,2023-05-19 20:11:06,s
1,89139,1000061,heart.1000,I,2023-06-17 11:42:18,s
2,89140,1000061,heart.1000,I,2023-06-17 11:43:19,s
3,67127,1000083,heart.777,I,2023-05-26 20:05:06,s
4,6322,1000145,heart.777,I,2023-05-14 17:42:04,s
...,...,...,...,...,...,...
44749,7659,999927,heart.1000,I,2023-05-14 19:36:54,s
44750,14838,999927,heart.1000,I,2023-05-15 17:15:33,s
44751,25976,999927,heart.1000,I,2023-05-16 23:48:47,s
44752,70917,999942,heart.777,I,2023-05-27 20:22:59,s


In [11]:
# 구매 실패 기록 index 확인용
total_history[total_history['is_success']=='f']

,index,user_id,productId,phone_type,created_at,is_success
408,17,1008906,NaN,I,2023-05-25 16:06:32,f
852,19,1016936,NaN,I,2023-05-18 22:55:31,f
1898,21,1039257,heart.777,A,2023-05-14 23:17:32,f
2787,22,1055891,heart.200,A,2023-05-14 14:49:22,f
2818,23,1056623,heart.777,A,2023-05-20 03:36:57,f
...,...,...,...,...,...,...
41315,4,894196,NaN,I,2023-06-05 00:01:04,f
41397,5,897014,NaN,I,2023-06-25 20:37:53,f
41988,6,917396,NaN,I,2023-06-03 20:21:33,f
42039,7,918859,NaN,I,2023-05-16 21:19:53,f


In [ ]:
# 구매하고 바로 실패한듯
# 구매부터 실패까지 2초밖에 안걸림
# 근데 왜 productId가 null이 떴지?

total_history.iloc[408 - 1: 408 + 1]

,index,user_id,productId,phone_type,created_at,is_success
407,62635,1008906,heart.200,I,2023-05-25 16:06:30,s
408,17,1008906,NaN,I,2023-05-25 16:06:32,f


In [12]:
# productId가 null이 안 뜬 기록들은 이전 기록 행이 없음
# 구매기록과 합쳐진건가?
total_history.iloc[2818 - 1: 2818 + 1]

,index,user_id,productId,phone_type,created_at,is_success
2817,91058,1056534,heart.1000,A,2023-07-20 20:27:42,s
2818,23,1056623,heart.777,A,2023-05-20 03:36:57,f


In [ ]:
cond = (total_history['productId'].notnull()) & (total_history['is_success'] == 'f')
not_null_idx = total_history[cond].index

total_df = pd.DataFrame({})
is_diff = []
for i in range(len(total_history)):
    if i in not_null_idx:
        df = total_history.iloc[i - 1: i + 1] 
        total_df = pd.concat([total_df, df]) # 실패기록의 이전행, 현재행 저장
        if df['user_id'].iloc[1] == df['user_id'].iloc[0]: # 이전행, 현재행의 user_id가 같다면
            is_diff.append(i) # 어느 위치인지 알려주기

# 실패 기록 이전, 현재의 user_id가 같은 친구들
# 총 실패기록 22개 중 2개 (약 9.1%)
is_diff

[2787, 20524]

In [31]:
# user_id가 같지만 
# 이전 구매 기록과 현재의 실패 기록의 productId가 서로 다름
# productId가 있는 애들은 구매&실패 기록이 잘 합쳐진 애들인 듯

i = 2787
total_history.iloc[i - 1: i + 1]

,index,user_id,productId,phone_type,created_at,is_success
2786,3950,1055891,heart.777,A,2023-05-14 14:48:05,s
2787,22,1055891,heart.200,A,2023-05-14 14:49:22,f


In [32]:
i = 20524
total_history.iloc[i - 1: i + 1]

,index,user_id,productId,phone_type,created_at,is_success
20523,23819,1297332,heart.777,A,2023-05-16 20:51:13,s
20524,90,1297332,heart.200,A,2023-05-26 22:23:59,f


In [ ]:
# null이 아닌 실패기록 모음
# 이전행과 해당행을 모아둠
total_df.head(10)

,index,user_id,productId,phone_type,created_at,is_success
1897,19131,1039177,heart.200,I,2023-05-16 00:37:35,s
1898,21,1039257,heart.777,A,2023-05-14 23:17:32,f
2786,3950,1055891,heart.777,A,2023-05-14 14:48:05,s
2787,22,1055891,heart.200,A,2023-05-14 14:49:22,f
2817,91058,1056534,heart.1000,A,2023-07-20 20:27:42,s
2818,23,1056623,heart.777,A,2023-05-20 03:36:57,f
3038,6750,1060892,heart.777,I,2023-05-14 18:14:47,s
3039,24,1060915,heart.777,A,2023-05-15 13:45:10,f
3959,87364,1078172,heart.200,I,2023-06-07 18:45:31,s
3960,26,1078215,heart.777,A,2023-05-18 21:56:57,f


In [ ]:
# null이 아닌 애들이 구매기록과 실패기록이 합쳐진 거라면
# null이 뜬 애들은 이전 구매기록과 같아야겠지?
# 이전 행, 현재 행의 user_id가 서로 같은지 확인해보자

cond = (total_history['productId'].isnull()) & (total_history['is_success'] == 'f')
null_idx = total_history[cond].index

total_df = pd.DataFrame({})
is_diff = []
for i in range(len(total_history)):
    if i in null_idx:
        df = total_history.iloc[i - 1: i + 1] 
        if df['user_id'].iloc[1] != df['user_id'].iloc[0]: # 이전행, 현재행의 user_id가 다르다면
            is_diff.append(i) # 어느 위치인지 알려주기

# 실패 기록 이전, 현재의 user_id가 다른 친구들
# 총 실패기록 63개 중 2개 (약 3.2%)
is_diff

[22537, 31579]

In [ ]:
# 얘네는 이전 행에 같은 유저의 구매 기록이 아예 없네
i = 22537
total_history.iloc[i - 1: i + 1]

,index,user_id,productId,phone_type,created_at,is_success
22536,45457,1322638,heart.777,A,2023-05-21 14:03:42,s
22537,98,1322662,NaN,I,2023-05-19 22:38:32,f


In [34]:
i = 31579
total_history.iloc[i - 1: i + 1]

,index,user_id,productId,phone_type,created_at,is_success
31578,86761,1445384,heart.777,I,2023-06-06 14:29:09,s
31579,129,1445414,NaN,I,2023-05-23 18:20:26,f


[일단 정리]
- 실패 기록 중 
- productId가 null이면, 이전 구매기록과 합쳐지지 않은 것으로 추정
    - 왜나면 해당 실패기록이 이전 구매기록과 같은 유저였음 (예외 기록이 약 3%)
- null이 아니면, 이전 구매기록과 합쳐진 것으로 추정
    - 왜나면 해당 실패기록이 이전 구매기록과 다른 유저였음 (예외 기록이 약 9%)
- 위의 예외 기록들은 어차피 이전 구매기록들이 없으니, 일단 제외하고 진행함 (총 4개)

In [65]:
# 구매부터 실패까지 평균 시간을 구해보자
# productId가 null이더라도
# 구매~실패 기록 차이가 최대 하루를 못 넘기면
# null값은 바로 이전 구매 기록의 productId이 조금 확실해지겠지

# 예외기록들은 제외
drop_idx = [2787, 20524, 22537, 31579]
new_total_history = total_history.drop(index=drop_idx)

# product_id가 null이 아닌 실패기록들도 제외
# 구매기록과 합쳐진 애들로 보기 때문
cond = (new_total_history['productId'].notnull()) & (new_total_history['is_success'] == 'f')
new_total_history = new_total_history[~cond]
new_total_history.reset_index(inplace=True)

new_total_history['created_at'] = pd.to_datetime(new_total_history['created_at']).dt.floor('s')

times = []
remove_history_idx = []

# null 친구들만 시간 차이를 계산함
# null 아닌 기록들은 이전 구매 기록의 user_id와 다르기 때문
for i in range(len(new_total_history)):
    if new_total_history.iloc[i]['is_success'] == 'f':
        if new_total_history.iloc[i]['user_id'] == new_total_history.iloc[i-1]['user_id']: # 실패 기록 이전 행이랑 user_id가 같다면
            time_diff = new_total_history.iloc[i]['created_at'] - new_total_history.iloc[i-1]['created_at']
            times.append(time_diff)
            remove_history_idx.append(i)    # 삭제할 이전행, 현재행 index 저장
            remove_history_idx.append(i-1)

# 이전행과 user_id가 같았던 실패기록들 (null 친구들)은 
# 총 63 - 2개(위의 예외기록 중 2개만 null이었음)로 61개였음
# times 개수와 잘 맞음
len(times) 

61

In [66]:
# 구매부터 취소까지 걸리는 평균
# 대략 24분 정도 걸림
np.mean(times)

Timedelta('0 days 00:23:58.754098360')

[정리]
- 예외 기록 4개를 제외하고 구매~취소 기간 차이를 계산한 결과, 약 24분으로 나옴
- 30분도 되지 않는 시간 내에 
- 같은 유저가 구매, 취소 기록을 동시에 갖고 있으므로
- <mark>null은 그저 구매 + 취소 기록이 합쳐지지 않은 것으로 추정할 수 있음      
---
‼️이 부분은 ai의 힘을 빌림, 그래서 그냥 그랬을 수도 있겠구나~ 하고 보면 될 듯   
[그럼 왜 기록이 안 합쳐졌을까]
- 로그 처리 순서나 트랜잭션 동기화 문제로 인해 구매-취소 로그가 매칭되지 못했을 가능성
- 이벤트 전달 타이밍 차이로 인해 일시적으로 productId가 비어 저장됐을 가능성
- 취소나 실패가 특정 조건에서 별도 로그로 남고, 그 과정에서 product 정보가 누락되도록 설계됐을 가능성

In [ ]:
# 구매-실패 행 제거한 최종 merge 버전
final_total_history = new_total_history.copy()
final_total_history.drop(index=remove_history_idx, inplace=True)
final_total_history.reset_index(drop=True, inplace=True)

# 실패 기록이 잘 사라진 걸 볼 수 있음
final_total_history[final_total_history['is_success'] == 'f']

,level_0,index,user_id,productId,phone_type,created_at,is_success


In [ ]:
# 상품명에 적힌 하트 수 컬럼명 생성
final_total_history['hearts'] = final_total_history['productId'].apply(lambda x: int(x.split('.')[-1]))

# 구매 기록의 총 하트 수 (취소 기록 제외)
# 취소기록 이전과 비교했을 때 상위권 변화가 없음
final_total_history.groupby('user_id')['hearts'].sum().reset_index(name='total_hearts').sort_values(by='total_hearts', ascending=False)

,user_id,total_hearts
4757,1141603,75977
5788,1167602,56777
8144,1216899,49977
3653,1111139,44777
2578,1081092,38977
...,...,...
7554,1205200,200
9566,1242808,200
9568,1242844,200
9050,1233129,200


### 최대 구매 하트수 이상 보유 중인 유저 분석 

In [77]:
# 그럼 최대 75,977개 이상의 하트를 가지고 있는 사람은 이상치인가?
# 몇 명 정도 존재하는 지 확인해야겠음 -> 13명 존재

over_hearts = hackle_merge['heart_balance'] > 75977
hackle_merge[over_hearts]['user_id'].nunique()

13

In [ ]:
# 이상치 유저 추출
outlier_user = hackle_merge[over_hearts]['user_id'].unique()

# 유저 정보 확인
# 분명 75,977개를 뛰어넘는 유저들의 정보를 가져왔는데
# 포인트가 1680, 8933개 등과 같이 매우 낮은 유저들 확인
# 유저 테이블은 2024년까지 업데이트 되어서인가? -> 그렇다기엔 만들어진 날짜가 2023년도임 -> 업데이트 날짜가 아니라 회원가입 날짜인가?
# 아니면 포인트와 하트는 아예 다른 게 확실한가?
accounts_user[accounts_user['user_id'].isin(outlier_user)]

,user_id,gender,point,friend_id_list,is_push_on,created_at,block_user_id_list,hide_user_id_list,ban_status,report_count,alarm_count,pending_chat,pending_votes,group_id,friend_count
11,833041,F,885000006,"[841345, 982531, 838785, 882567, 832520, 87949...",1,2023-03-31 23:32:36,[],[],N,1,8,0,0,149.0,67
692,838541,F,100404,"[841345, 982531, 843015, 879496, 832520, 84429...",1,2023-04-20 01:36:39,[],[],N,178,21442,4105,28,9.0,47
8926,849763,M,9070657,"[841345, 982531, 843015, 832520, 843017, 84429...",1,2023-04-28 19:13:42,[],[],N,0,6,1,3,78425.0,44
64041,914589,M,274082,"[971648, 1428614, 867469, 863503, 913940, 1240...",0,2023-05-06 22:04:31,[],[],N,0,0,0,0,8541.0,31
113137,967442,F,8933,"[902144, 1284225, 936196, 1012359, 898445, 907...",1,2023-05-07 21:56:51,[],[],N,0,0,0,0,10396.0,27
407467,1287325,M,57667,"[1563520, 1183746, 1288196, 1290502, 1575689, ...",1,2023-05-15 19:33:08,[],[],N,0,1,0,38,51364.0,70
410358,1290502,M,135176,"[1563520, 1183746, 1288196, 1575689, 1406091, ...",1,2023-05-15 21:10:44,[],[],N,0,2,0,0,51364.0,56
537731,1431519,M,62272,"[1563520, 1183746, 1288196, 1290502, 1575689, ...",1,2023-05-21 22:27:30,[],[],N,0,1,0,134,70743.0,47
543377,1437875,M,9988541,"[1150464, 1197319, 1579656, 1494535, 1058956, ...",1,2023-05-22 04:55:08,[832740],[],N,1,201,3,17,1.0,86
658585,1563520,M,206081,"[1183746, 1288196, 1290502, 1575689, 1191690, ...",1,2023-06-02 21:38:12,[],[],N,0,2,0,6,82805.0,50


In [82]:
# 75,977보다 낮은 포인트를 가지고 있는 유저들 hackle 확인
outlier_user_info = accounts_user[accounts_user['user_id'].isin(outlier_user)]
under_outlier_info = outlier_user_info[outlier_user_info['point'] < 75977]
under_outlier_info

,user_id,gender,point,friend_id_list,is_push_on,created_at,block_user_id_list,hide_user_id_list,ban_status,report_count,alarm_count,pending_chat,pending_votes,group_id,friend_count
113137,967442,F,8933,"[902144, 1284225, 936196, 1012359, 898445, 907...",1,2023-05-07 21:56:51,[],[],N,0,0,0,0,10396.0,27
407467,1287325,M,57667,"[1563520, 1183746, 1288196, 1290502, 1575689, ...",1,2023-05-15 19:33:08,[],[],N,0,1,0,38,51364.0,70
537731,1431519,M,62272,"[1563520, 1183746, 1288196, 1290502, 1575689, ...",1,2023-05-21 22:27:30,[],[],N,0,1,0,134,70743.0,47
671665,1577930,M,1680,"[838785, 982531, 882567, 879496, 838541, 83649...",1,2023-07-09 04:39:15,[],[],N,0,2,0,31,1.0,46


In [83]:
under_outlier_user = under_outlier_info['user_id'].unique()
under_outlier_hackle = hackle_merge[hackle_merge['user_id'].isin(under_outlier_user)].sort_values(by=['user_id', 'event_datetime'])
under_outlier_hackle

,session_id,user_id,language,osname,osversion,versionname,device_id,event_id,event_datetime,event_key,item_name,page_name,friend_count,votes_count,heart_balance,question_id
6022766,4LxbU2CW2ZfGL6jQYT1K9yRpVTG3,1287325,ko-KR,iOS,16.6,2.0.5,BB3D3B32-EAD1-40F6-A2A8-BB5A98FAB9E0,06aae2b1-afa1-4798-a7ed-756374e737fe,2023-07-19 00:06:01,$session_start,NaN,NaN,69.0,342.0,88877.0,NaN
6022863,4LxbU2CW2ZfGL6jQYT1K9yRpVTG3,1287325,ko-KR,iOS,16.6,2.0.5,BB3D3B32-EAD1-40F6-A2A8-BB5A98FAB9E0,8aada421-b178-4ab5-a2f6-b724ed3f6b94,2023-07-19 00:06:01,launch_app,NaN,NaN,69.0,342.0,88877.0,NaN
12627712,4LxbU2CW2ZfGL6jQYT1K9yRpVTG3,1287325,ko-KR,iOS,16.5,2.0.3,BB3D3B32-EAD1-40F6-A2A8-BB5A98FAB9E0,06aae2b1-afa1-4798-a7ed-756374e737fe,2023-07-19 00:06:01,$session_start,NaN,NaN,69.0,342.0,88877.0,NaN
12627809,4LxbU2CW2ZfGL6jQYT1K9yRpVTG3,1287325,ko-KR,iOS,16.5,2.0.3,BB3D3B32-EAD1-40F6-A2A8-BB5A98FAB9E0,8aada421-b178-4ab5-a2f6-b724ed3f6b94,2023-07-19 00:06:01,launch_app,NaN,NaN,69.0,342.0,88877.0,NaN
12797541,4LxbU2CW2ZfGL6jQYT1K9yRpVTG3,1287325,ko-KR,iOS,16.5,2.0.5,BB3D3B32-EAD1-40F6-A2A8-BB5A98FAB9E0,06aae2b1-afa1-4798-a7ed-756374e737fe,2023-07-19 00:06:01,$session_start,NaN,NaN,69.0,342.0,88877.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15306284,Y7rvLcfCMBWCGlGqguDr59rNQoL2,967442,ko,Android,13,2.0.5,61001a1f-7b8b-4fce-94be-6f3c626b38fe,15633acb-baa4-4d4b-9e87-5c5cb2502bcf,2023-08-05 18:29:17,view_timeline_tap,NaN,NaN,28.0,161.0,8517.0,NaN
15306286,Y7rvLcfCMBWCGlGqguDr59rNQoL2,967442,ko,Android,13,2.0.5,61001a1f-7b8b-4fce-94be-6f3c626b38fe,31eb647a-0038-42e9-b865-a4caa114eeb9,2023-08-05 18:29:17,click_bottom_navigation_questions,NaN,NaN,28.0,161.0,8517.0,NaN
15306287,Y7rvLcfCMBWCGlGqguDr59rNQoL2,967442,ko,Android,13,2.0.5,61001a1f-7b8b-4fce-94be-6f3c626b38fe,50930719-5d43-4646-b2f2-60a1ab137464,2023-08-05 18:29:17,view_lab_tap,NaN,NaN,28.0,161.0,8517.0,NaN
15306296,Y7rvLcfCMBWCGlGqguDr59rNQoL2,967442,ko,Android,13,2.0.5,61001a1f-7b8b-4fce-94be-6f3c626b38fe,e8abeda3-7814-41a4-9e97-3b0bb2bb4c43,2023-08-05 18:29:17,view_questions_tap,NaN,NaN,28.0,161.0,8517.0,NaN


In [ ]:
# 아니 이 유저는
# 처음 로그인할 때는 8천대였다가
# 갑자기 하트가 7억개로 갑자기 뛰네
# 그러다 세션 id가 바뀌면서 다시 8천대로 돌아옴
# 7/21 ~ 8/5까지 7억개로 지냈음

colums = ['session_id', 'user_id', 'device_id' ,'event_datetime', 'event_key', 'friend_count', 'votes_count', 'heart_balance']
under_outlier_hackle[under_outlier_hackle['user_id'] == '967442'][colums]

,session_id,user_id,device_id,event_datetime,event_key,friend_count,votes_count,heart_balance
13435204,b653e4fe-f7a4-45ba-be46-7f01053b6c21,967442,b653e4fe-f7a4-45ba-be46-7f01053b6c21,2023-07-21 01:53:30,$session_end,NaN,NaN,NaN
13435197,b653e4fe-f7a4-45ba-be46-7f01053b6c21,967442,b653e4fe-f7a4-45ba-be46-7f01053b6c21,2023-07-21 02:24:36,launch_app,NaN,NaN,NaN
13435201,b653e4fe-f7a4-45ba-be46-7f01053b6c21,967442,b653e4fe-f7a4-45ba-be46-7f01053b6c21,2023-07-21 02:24:36,$session_end,NaN,NaN,NaN
13435208,b653e4fe-f7a4-45ba-be46-7f01053b6c21,967442,b653e4fe-f7a4-45ba-be46-7f01053b6c21,2023-07-21 02:24:36,$session_start,NaN,NaN,NaN
13435195,b653e4fe-f7a4-45ba-be46-7f01053b6c21,967442,b653e4fe-f7a4-45ba-be46-7f01053b6c21,2023-07-21 02:24:37,$session_start,28.0,161.0,8517.0
13435187,b653e4fe-f7a4-45ba-be46-7f01053b6c21,967442,b653e4fe-f7a4-45ba-be46-7f01053b6c21,2023-07-21 02:24:56,click_bottom_navigation_profile,15.0,28.0,703126260.0
13435192,b653e4fe-f7a4-45ba-be46-7f01053b6c21,967442,b653e4fe-f7a4-45ba-be46-7f01053b6c21,2023-07-21 02:24:56,view_lab_tap,15.0,28.0,703126260.0
13435206,b653e4fe-f7a4-45ba-be46-7f01053b6c21,967442,b653e4fe-f7a4-45ba-be46-7f01053b6c21,2023-07-21 02:24:56,view_timeline_tap,15.0,28.0,703126260.0
13435202,b653e4fe-f7a4-45ba-be46-7f01053b6c21,967442,b653e4fe-f7a4-45ba-be46-7f01053b6c21,2023-07-21 02:25:01,click_appbar_setting,15.0,28.0,703126260.0
13435188,b653e4fe-f7a4-45ba-be46-7f01053b6c21,967442,b653e4fe-f7a4-45ba-be46-7f01053b6c21,2023-07-21 02:25:25,click_appbar_setting,15.0,28.0,703126260.0
